#### Imports

In [4]:
import os

import numpy as np
import cv2 as cv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, transforms

from model.load_model import load_model

#### Variables

In [5]:
train_data_path = 'data/train/'
test_data_path = 'data/test/'
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
image_size = 224

#### Dataset definition

In [6]:
# class HotdogDataset(Dataset):
#     def __init__(self, data_path: str) -> None:
#         data = []
#         current_folder = os.path.join(data_path, 'hotdog')
#         for file_name in os.listdir(current_folder):
#             file_path = os.path.join(current_folder, file_name)
#             data.append([file_path, 1])  # 1 for being hotdog
        
#         current_folder = os.path.join(data_path, 'nothotdog')
#         for file_name in os.listdir(current_folder):
#             file_path = os.path.join(current_folder, file_name)
#             data.append([file_path, 0])  # 0 for being not hotdog
        
#         self.data = np.asarray(data)
#         np.random.shuffle(self.data)
#         self._n_samples = len(data)
    
#     def __len__(self) -> int:
#         return self._n_samples
    
#     def __getitem__(self, idx: int) -> tuple:
#         image, label = self.data[idx]
#         image = cv.imread(image)
#         image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
#         image = cv.resize(image, (image_size, image_size))
#         image = np.moveaxis(image, 2, 0)

#         if label:
#             label = [0, 1]
#         else:
#             label = [1, 0]

#         return torch.from_numpy(image).float(), torch.tensor(label).float()

#### PyTorch dataset loading

In [7]:
train_transforms = transforms.Compose([transforms.Resize((image_size, image_size)),
                                       transforms.ToTensor()
])
test_transforms = transforms.Compose([transforms.Resize((image_size, image_size)),
                                      transforms.ToTensor(),
])


train_data = datasets.ImageFolder(train_data_path, transform=train_transforms)
test_data = datasets.ImageFolder(test_data_path, transform=test_transforms)


#### DataLoaders

In [8]:
# train_dataloader = DataLoader(HotdogDataset(train_data_path), batch_size=8, shuffle=True)
# test_dataloader = DataLoader(HotdogDataset(test_data_path), batch_size=8, shuffle=True)

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

#### Model loading

In [9]:
model_params = {
    'model_ver': 0,
    'in_channels': 3,
    'image_size': image_size,
    'num_classes': 2,
    'block_types': ['C', 'C', 'T', 'T'],
}

model = load_model(**model_params)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model.to(device)

c:\Users\arax\.conda\envs\general\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


CoAtNet(
  (s0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate=none)
    )
  )
  (s1): Sequential(
    (0): MBConv(
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (proj): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): PreNorm(
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fn): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

#### Train (incomplete)

In [10]:
model.to(device)
epochs = 5

for epoch in range(epochs):
    for data, label in train_dataloader:
        data, label = data.to(device), label.to(device)

        outputs = model(data)
        loss = criterion(outputs, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'[EPOCH {epoch+1:3d}]    loss: {loss.item()}')


[EPOCH   1]    loss: 0.8132617473602295
[EPOCH   2]    loss: 0.8132617473602295
